# Name: Frank Ho
### Date: 08/14/2023

In [ ]:
import pandas as pd

awards = pd.read_csv("awards_data.csv")
player_data = pd.read_csv("player_stats.csv")
team_data = pd.read_csv("team_stats.csv")
rebounding_data = pd.read_csv("team_rebounding_data_22.csv")

## Part 1 -- Awards  


In [ ]:
def find_avg_ppg(col_to_use, truth_val):
    """Function to find the average ppg based on a column: col_to_use and a certain value: truth_val
    Args:
        col_to_use (str): Column will be either 'All NBA First Team', 'All NBA Second Team',
            'All NBA Third Team', or 'all_star_game'
        truth_val (1 or True): will be 1 for 'All NBA First Team', 'All NBA Second Team', or
            'All NBA Third Team', and True for 'all_star_game'
            
    Returns: 
        (int): rounded, to 1 decimal place, for average point per season for the 2007-2021 seasons
    """
    # Gets only rows for season between 2007 and 2021 season
    all_nba_or_allstar_awards = awards[(awards['season'] >= 2007) & (awards['season'] <= 2021)]
    # Gets only rows where the col_to_use value is the truth_val
    all_nba_or_allstar_awards = all_nba_or_allstar_awards.loc[awards[col_to_use] == truth_val]
    # Left Join on 'season' and 'nbapersonid' col
    all_nba_or_allstar = all_nba_or_allstar_awards.merge(player_data[['season', 'nbapersonid','points','games']], how='left', on=['season', 'nbapersonid'])
    # Make a new column to calculate points per game for each player
    all_nba_or_allstar['ppg'] = all_nba_or_allstar['points'] / all_nba_or_allstar['games'] 
    # Calculate the average points per game
    avg_ppg = all_nba_or_allstar['ppg'].mean()
    # Round to the nearest 1 decimal place
    rounded_avg_pts = round(avg_ppg, 1)
    return rounded_avg_pts

In [ ]:
first_team_avg_ppg = find_avg_ppg('All NBA First Team', 1)
print(f"1st Team: {first_team_avg_ppg} points per game")

second_team_avg_ppg = find_avg_ppg('All NBA Second Team', 1)
print(f"2nd Team: {second_team_avg_ppg} points per game")

third_team_avg_ppg = find_avg_ppg('All NBA Third Team', 1)
print(f"3rd Team: {third_team_avg_ppg} points per game")

allstar_avg_ppg = find_avg_ppg('all_star_game', True)
print(f"All-Star: {allstar_avg_ppg} points per game")

<strong><span style="color:red">ANSWER 1:</span></strong>   

1st Team: XX.X points per game  
2nd Team: XX.X points per game  
3rd Team: XX.X points per game  
All-Star: XX.X points per game  

In [ ]:
# Get a sorted dataframe first by season and then nbapersonid in ascending order
sorted_awards = awards.sort_values(by=['season','nbapersonid'])
# Get rid fo all players didn't make it to any all nba teams
only_all_team_awards = sorted_awards[
    (sorted_awards['All NBA First Team'] == 1) |
    (sorted_awards['All NBA Second Team'] == 1) |
    (sorted_awards['All NBA Third Team'] == 1)
]
# Get the 1st occurence of making an All Team by dropping duplicated values based on the 'nbapersonid' column
first_occ_all_team = only_all_team_awards.drop_duplicates(subset='nbapersonid')
# Left merge with first_occ_all_team and player_data dataframes on 'season' and 'nbapersonid'
first_occ_all_team = first_occ_all_team.merge(player_data[['season', 'nbapersonid', 'draftyear']], how='left', on=['season', 'nbapersonid'])
# Calculate the years it took the player to make an All Team (add 1 to account for the first season)
first_occ_all_team['years_for_all_nba'] = first_occ_all_team['season'] - first_occ_all_team['draftyear'] + 1
# Calculate the average years it took to make an All Team
avg_years_all_team = first_occ_all_team['years_for_all_nba'].mean()
# Round to the nearest 1 decimal place
rounded_avg_years_all_team = round(avg_years_all_team, 1)
print(f'{rounded_avg_years_all_team} Years')

<strong><span style="color:red">ANSWER 2:</span></strong>  

XX.X Years  

## Data Cleaning Interlude  


In [ ]:
import numpy as np

# Grab only players with a draft year of 2010
player_data_2010 = player_data[player_data['draftyear'] == 2010]
# Change 'games' column name to 'player_games'
player_data_2010.rename(columns={'games': 'player_games'}, inplace=True)
# LEFT Join to team_data 
player_data_2010 = player_data_2010.merge(team_data[['nbateamid', 'season', 'games']], how='left', on=['nbateamid', 'season'])
# Calculate the number of games started in case the season had less than 82 games
player_data_2010['calc_started'] = (player_data_2010['games_start'] * (82 / player_data_2010['games'])).round()
# Calculate the number of minutes played in case the season had less than 82 games
player_data_2010['calc_mins'] = (player_data_2010['mins'] * (82 / player_data_2010['games'])).round()

# Grab only the data with season greater than 2015, we only want to check career in the 5th year foward
awards_gt_2010 = awards[awards['season'] >= 2015]
# LEFT JOIN to get the calc_started games and calc_mins played
awards_gt_2010 = awards_gt_2010.merge(player_data_2010[['season', 'nbapersonid', 'calc_started', 'calc_mins']], how='left', on=['season', 'nbapersonid'])
# Dropping all rows without calc_started, essentially dropping all players not drafting in 2010
awards_gt_2010.dropna(subset=['calc_started'], inplace=True)

# get a sum of All NBA awards for each row
col_to_sum = ['All NBA Defensive First Team', 'All NBA Defensive First Team', 'All NBA First Team', 'All NBA Second Team', 'All NBA Third Team']
conditions = [
    # if it's greater than 1, than the person is 'Elite', or if the person is ranked 1 in DPOY or MVP
    (awards_gt_2010[col_to_sum].sum(axis=1) > 0) | (awards_gt_2010['Defensive Player Of The Year_rk'] == 1) | (awards_gt_2010['Most Valuable Player_rk'] == 1),
    # If the person was in the all start game
    awards_gt_2010['all_star_game'] == True,
    # If the calc_started games is greater than 41 or if calc_min is greater than 2000
    (awards_gt_2010['calc_started'] >= 41) | (awards_gt_2010['calc_mins'] >= 2000),
    awards_gt_2010['calc_mins'] >= 1000,
    awards_gt_2010['calc_mins'] >= 1,
    # Edge case of players who were on the team, but didn't play a single minute
    awards_gt_2010['calc_mins'] == 0
]
# Based on the conditions above, these are your given choices in their respective order
choices = ['Elite', 'All-Star', 'Starter', 'Rotation', 'Roster', 'Bench']
# Out of League is the default if the other conditions are not met
awards_gt_2010['season_outcome'] = np.select(conditions, choices, default='Out of the League')
# Create a table of the count, group by 'nbapersonid', take the 'season_outcome' columns and aggregate by count
players_career_outcome = awards_gt_2010.pivot_table(index='nbapersonid', columns='season_outcome', aggfunc='size', fill_value=0)
players_career_outcome['check_out_of_league'] = players_career_outcome.sum(axis=1)
out_of_league_count = (players_career_outcome['check_out_of_league'] <= 1).sum()
players_career_outcome = players_career_outcome[players_career_outcome['check_out_of_league'] > 1]

importance_outcomes = ['Elite', 'All-Star', 'Starter', 'Rotation', 'Roster']

np_arr = players_career_outcome.values
# Go through the np_arr row by row
for row in np_arr:
    # For each row, check each column by order of importance
    for col in importance_outcomes:
        col_idx = players_career_outcome.columns.get_loc(col)
        # If the total count of the row is 2 or greater (2 or more seasons)
        if row[col_idx] >= 2:
            # Then set that row value to 1
            row[col_idx] = 1
            # Set all other values to 0
            row[:col_idx] = 0
            row[col_idx+1:] = 0
            # break out of the inner for loop, so we don't waste time checking all other columns
            break
# After modifications are made to np_arr, change all rows and columns in players_career_outcome to match
players_career_outcome.iloc[:, :] = np_arr     
# Create a df with the count of each career outcome total for all players in 2015
career_outcome = players_career_outcome.sum()
output = '\n'.join([f"{col}: {career_outcome[col]} players." for col in importance_outcomes]) + f'\nOut of League: {out_of_league_count} players.' 
print(output)

<strong><span style="color:red">ANSWER 3:</span></strong>  

Elite: X players.  
All-Star: X players.  
Starter: X players.  
Rotation: X players.  
Roster: X players.  
Out of League: X players.  

1. Transform the data to have a player's average points per game and total appearances in All-Star Games. (Only include players drafted on or before 2015)

In [ ]:
# From the player_data df, only grab players who were drafted in or before 2015
player_data_lt_2015 = player_data[player_data['season'] <= 2015]
awards_data_lt_2015 = awards.merge(player_data_lt_2015[['season', 'nbapersonid','points','games']], how='left', on=['season', 'nbapersonid'])
awards_data_lt_2015['ppg'] = awards_data_lt_2015['points'] / awards_data_lt_2015['games'] 
awards_data_lt_2015.dropna(subset=['ppg'], inplace=True)
# fill na with False to avoid any errors later
awards_data_lt_2015['all_star_game'] = awards_data_lt_2015['all_star_game'].fillna(False).astype(bool)
# Average the players' across the 2015 season grouped by 'nbapersonid'
avg_ppg_per_player = awards_data_lt_2015.groupby('nbapersonid')['ppg'].mean()
# Sum the the players' appearances in all_star_games grouped by 'nbapersonid'
tot_all_star_games_per_player = awards_data_lt_2015.groupby('nbapersonid')['all_star_game'].sum()
# merge the 2 aggregate dfs by 'nbapersonid'
lt_2015_data = pd.merge(avg_ppg_per_player, tot_all_star_games_per_player, on='nbapersonid')

2. Check the shape

In [ ]:
lt_2015_data.head()

3. Set up the dependent (all_star_game) and independent variables (ppg)

In [ ]:
# Looks the same as step 2, but this would compensate if there are any other columns and only grab these
data_ = lt_2015_data.loc[:,['ppg','all_star_game']]
data_.head(5)

4. Simple Linear Regression Graph

In [ ]:
import matplotlib.pyplot as plt 
lt_2015_data.plot(x='ppg', y='all_star_game', style='o')
plt.xlabel('Points Per Game')
plt.ylabel('Total All-Star Games')
plt.show()

5. Divide the data into dependent and independent variables

In [ ]:
X = pd.DataFrame(lt_2015_data['ppg'])
y = pd.DataFrame(lt_2015_data['all_star_game'])

6. Split the data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

7. Shape of train and test sets

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

8. Train the algorithm

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

9. Create a column to predict if the player would be an All-Star based on if they had a total of 2 or more All-Star appearances.

In [ ]:
all_star_threshold = 2.0
is_all_star = y_pred >= all_star_threshold
lt_2015_data.loc[X_test.index, 'y_pred'] = y_pred
lt_2015_data['actual_all_star'] = lt_2015_data['all_star_game'] >= 2
lt_2015_data['predicted_all_star'] = False
lt_2015_data.loc[X_test.index, 'predicted_all_star'] = is_all_star

10. Check dataframe

In [ ]:
lt_2015_data.head()

11. Evaluate the Algorithm

In [ ]:
from sklearn import metrics
import numpy as np
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

12. Make a prediction on players drafted between 2018-2021

In [ ]:
player_data_2018_2021 = player_data[(player_data['season'] >= 2018) & (player_data['season'] <= 2021)]
awards_data_2018_2021  = awards.merge(player_data_2018_2021[['season', 'nbapersonid','points','games']], how='left', on=['season', 'nbapersonid'])
awards_data_2018_2021['ppg'] = awards_data_2018_2021 ['points'] / awards_data_2018_2021['games'] 
awards_data_2018_2021 = awards.merge(player_data_2018_2021[['season', 'nbapersonid','points','games']], how='left', on=['season', 'nbapersonid'])
awards_data_2018_2021['ppg'] = awards_data_2018_2021['points'] / awards_data_2018_2021['games'] 
awards_data_2018_2021.dropna(subset=['ppg'], inplace=True)
awards_data_2018_2021['all_star_game'] = awards_data_2018_2021['all_star_game'].fillna(False).astype(bool)
ppg_per_player = awards_data_2018_2021.groupby('nbapersonid')['ppg'].mean()
tot_all_star_2018_2021 = awards_data_2018_2021.groupby('nbapersonid')['all_star_game'].sum()

lt_2018_2021 = pd.merge(ppg_per_player, tot_all_star_2018_2021, on='nbapersonid')

13. Check the df

In [ ]:
lt_2018_2021.head()

14. Predict All-Start Game count with a threshold of 2 now.

In [ ]:
pred_2018_2021 = regressor.predict(lt_2018_2021[['ppg']])
all_star_thresh = 2.0
is_all_start_2018_2021 = pred_2018_2021 >= all_star_thresh
lt_2018_2021['predicted_all_star'] = is_all_start_2018_2021
lt_2018_2021['actual_all_star'] = lt_2018_2021['all_star_game'] >= 2

15. Check with added 'predicted_all_start' column

In [ ]:
lt_2018_2021.head()

16. Find the ids of player_name1, player_name2, player_name3

In [ ]:
players = ['player_name1', 'player_name2', 'player_name3']
rows_with_players = player_data.loc[player_data['player'].isin(players), ['nbapersonid', 'player']]
rows_with_players.drop_duplicates(subset=['nbapersonid', 'player'], inplace=True)

17. Get the resulting dataframe

In [ ]:
rows_with_players.head()

18. Merge with lt_2018_2021' to get the prediction

In [ ]:
rows_with_players.merge(lt_2018_2021, how='left', on='nbapersonid')

Final Thoughts
1. Because I don't have "formal" modeling experience, I took the suggestion of utilizing a simple linear regression approach with an averaged points per game across the the defined seasons as the independent variable and whether they got into the All-star game or not. If they had an outcome of 2 or more All-Star appearances across their career in the defined time-frame, then they would have had a career trajectory of an All-Star. Obviously, there are more factors, and maybe a multiple linear regression approach utilizing multiple statistics would have given a more accurate choice. An example would be a player who had a rebounded average of 20+ per game but only 5 points per game might also be a good candidate for an All-Star. But, I went with an average points per game because the audience tends to favor high scorers in voting. There are several other flaws with picking an average points per game across multiple seasons. For example, a player could average 30+ ppg across 2 seasons and less than 10 ppg which could bring their ppg below the threshold to be predicted an All-star, but in reality, the two 30+ seasons could have nabbed them two All-Star spots in actuality. Obviously, this model only predicts if the person will be an All-Star or not and exclude the other career outcomes. Similar to the method I utilized in the "Data Cleaning Interlude," I would clean the data into separate dataframes based on the factors that would have determine the other career outcomes. Then, utilize a multiple linear regression (or just the ppg as a single linear regression) to figure out if the player would have been "Elite" or any of the other career outcomes. Obviously, there would be True or False across the career outcomes. So in the final table, the player would have True or False for each of the career outcomes, and whatever the first True boolean value is across the career outcomes in order of importance would determine their actual result.
2. To fill in the missing data for All-NBA or All Rookie teams, I would use pandas interpolate() method. Interpolation utilizes known data to fill in the missing data, so maybe a linear interpolation.
6.
    - This model works by calculating if a person will be an All-Star based on their averaged points per game across multiple seasons. If the person has two All-Star appearances in their career, they will have had been considered an All-Star in their career trajectory. 
    
    First, we filter out the two excel files of data from 'player_stats' and 'awards_data' to only include the draft years that we defined. From then on, we created a column to house the points per game of each player by dividing all values in the points column from all values in the game column. By merging the two tables together, we can then calculate the average points per game across multiple seasons and also sum up the total All-Star games and lets create a table with just these two columns. As you can see the graph, the X variable, or horizontal line is our averaged points per game across multiple seasons. The Y variable, or vertical line shows our total All-Star appearances. As you can see there tends to be more All-Star appearances when the player averages more points. So, our Y variable is also the dependent variable because it depends on the X variable. From there, we take the the availabe ppg and total all-star appearances and randomly split the data with 80% that will be used for training, and 20% that will be used for testing. This is the data already available but only 80% will be trained by the model. During the training phase, patterns are analyzed to create some sort of algorithm or function that would calculate predicted values. The second phase is the testing phase, in which we input the 20% of data into the algorithm or function from the training phase to get predicted values. Finally, we can assess the model accuracy and precision by using formulas to show us how far or close the predicted values are from the actual values.
    - I mentioned strengths and weaknesses of the model in part 1.
    - I also address how I would compensate for the strengths and weakness of the model in part 1.


## Part 2 -- Predicting Team Stats  


In [ ]:
okc_rebounding_data_1_80 = rebounding_data[(rebounding_data['team'] == 'OKC') & 
                                           (rebounding_data['game_number'] >= 1) & 
                                           (rebounding_data['game_number'] <= 80)]
tot_off_reb = okc_rebounding_data_1_80['offensive_rebounds'].sum()
tot_off_reb_chances = okc_rebounding_data_1_80['off_rebound_chances'].sum()
pred_game_81_percent = tot_off_reb / tot_off_reb_chances * 100

print("{:.1f}%".format(pred_game_81_percent))

<strong><span style="color:red">ANSWER 1:</span></strong>  

XX.X% 

## 1. Home/Away games might cause ability to rebound better or worse
## 2. Facing a strong (better winning percentage or great defensive rebounding) vs weak opponent
## 3. Distance of opponent
### WEAKNESSES
We know that generally teams will do better in home games vs away games. This could be due in large part to many factors, including pressure of the crowd on away teams and
preference of referees to make foul calls against the away team and ignore calling fouls for home team. Facing stronger opponents or ones with better defensive rebounding 
might also affect offensive rebounding as the players will have to fight the opponent for the rebound. Lastly, long flights vs short flights might cause players to be tired
and not as fresh for games.

### ANSWER
To account for this calculate a ```weight_score``` by assigning a lower weight to home games (ex. 0.8) and higher weight to away games (ex. 1.2). This is to account for the probability that it's easier to rebound during home games vs away games. The same can be done when 
facing teams of higher win rate vs lower win rate. Finally, we could calculate another weight depending on conference or even division. For example, a west coast team 
would have a lower weight when facing a west coast team because they don't have to travel as far vs higher weight when facing an east coast team. Take all these weights,
and multiple them together for the ```weight_score```. Lastly, the weighted offensive rebound percentage would be simply just take the offensive rebound percetange and
multiply that by the ```weight_score```.